# Instalando dependências

In [ ]:
from IPython.display import clear_output
import sys

IN_COLAB = 'google.colab' in sys.modules

In [ ]:
if IN_COLAB:
    !git clone https://github.com/LucaLemos/UFRPE_AprendizagemReforco
    sys.path.append("/content/UFRPE_AprendizagemReforco")

    clear_output()
else:
    from os import path
    sys.path.append( path.dirname( path.dirname( path.abspath("__main__") ) ) )

In [ ]:
if IN_COLAB:
    # for saving videos
    !apt-get install ffmpeg
    !pip install gymnasium==1.0.0   # conferir se precisa
    #!pip install tianshou # Para criar o Replay_Buffer
    #!pip install d3rlpy==2.7.0
    # clone repository

# Criando Dataset

In [1]:
import gymnasium as gym
import torch
from util.algorithms import run_sarsa
from util.network import ReplayBuffer, MLP, epsilon_greedy_qnet, calc_loss
from IPython.display import clear_output

In [2]:
DATASET_SIZE = 200_000  # Tamanho do conjunto de dados (replay buffer)
LEARNING_RATE = 1e-3  # Taxa de aprendizado para o otimizador
GAMMA = 0.99  # Fator de desconto
BATCH_SIZE = 128  # Tamanho do batch para treinamento da rede neural

In [3]:
ENV_NAMES = []
ENVS_REPLAY_BUFFER = []

## Dataset Básico

In [4]:
# Passo 1: Coletar um conjunto fixo de transições (Replay Buffer)
# Ambientes Básicos
ENV_NAMES += ["FrozenLake-v1", "Taxi-v3", "CliffWalking-v0"]
for i, env_name in enumerate(ENV_NAMES):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    replay_buffer = ReplayBuffer(DATASET_SIZE, BATCH_SIZE, device)
    env = gym.make(env_name, render_mode="rgb_array")
    
    sum_rewards_per_ep, q = run_sarsa(env, replay_buffer, DATASET_SIZE, LEARNING_RATE, GAMMA)
    ENVS_REPLAY_BUFFER.append((env_name, env, replay_buffer, sum_rewards_per_ep, q))
    replay_buffer.save_config(f"config\dataset\sarsa\{env_name}.json")


Run sarsa in <TimeLimit<OrderEnforcing<PassiveEnvChecker<FrozenLakeEnv<FrozenLake-v1>>>>>
OI
Episódio 0 terminou com recompensa 0.0 na transição 34
Episódio 100 terminou com recompensa 0.0 na transição 1415
Episódio 200 terminou com recompensa 0.0 na transição 3061
Episódio 300 terminou com recompensa 0.0 na transição 4765
Episódio 400 terminou com recompensa 0.0 na transição 6301
Episódio 500 terminou com recompensa 0.0 na transição 7886
Episódio 600 terminou com recompensa 0.0 na transição 9323
Episódio 700 terminou com recompensa 0.0 na transição 10850
Episódio 800 terminou com recompensa 0.0 na transição 12336
Episódio 900 terminou com recompensa 0.0 na transição 13921
Episódio 1000 terminou com recompensa 0.0 na transição 15295
Episódio 1100 terminou com recompensa 0.0 na transição 16793
Episódio 1200 terminou com recompensa 0.0 na transição 18500
Episódio 1300 terminou com recompensa 0.0 na transição 19607
Episódio 1400 terminou com recompensa 0.0 na transição 21049
Episódio 1500

## Dataset com Rede Neural

In [5]:
from torch import optim, nn
import numpy as np

In [6]:
def DQN_TRAIN(env, env_name, gamma, qnet, qnet_lr, target_qnet, target_update_freq, replay_buffer, batch_size, epsilon_f, epsilon_decay_period, transitions):
    # Cria o otimizador, que vai fazer o ajuste dos pesos da 'qnet',
    # Usa uma técnica de gradiente descendente de destaque, chamada ADAM
    optimizer = optim.Adam(qnet.parameters(), lr=qnet_lr)

    episode_reward_list = []
    step = 0
    epsilon = 1.0

    state, _ = env.reset()
    episode_reward = 0.0

    for _ in range(transitions):
        # Decaimento linear do epsilon
        epsilon = max(epsilon_f, 1.0 - step / epsilon_decay_period)

        action = epsilon_greedy_qnet(qnet, env, state, epsilon)

        # Faz um passo / Aplica uma ação no ambiente
        new_state, reward, terminated, truncated, _ = env.step(action)

        step += 1
        done = terminated or truncated
        episode_reward += reward

        # Adiciona no buffer
        replay_buffer.add(state, action, reward, terminated, new_state)
        state = new_state

        if done:
            episode_reward_list.append(episode_reward)
            if len(episode_reward_list) % 100 == 0:
                mean_reward = np.mean(episode_reward_list[-100:])
                print(f'Episode Reward: {episode_reward_list[-1]} | Epsilon: {epsilon} | Step: {step} | Mean Reward: {mean_reward}')
            state, _ = env.reset()
            episode_reward = 0.0

            # Abaixo, faz vários loggings de dados
            
        # Faz a 'tgt_net' receber os mesmos valores de pesos da 'qnet', na frequência indicada
        
        if step > batch_size:
            if step % target_update_freq == 0:
                target_qnet.load_state_dict(qnet.state_dict())
            # Escolhe amostras aleatórios do buffer e faz uma atualização dos pesos da rede
            optimizer.zero_grad()
            batch = replay_buffer.sample_continuous()
            loss_t = calc_loss(batch, qnet, target_qnet, gamma)
            loss_t.backward()
            optimizer.step()
            
    mean_reward = np.mean(episode_reward_list[-100:])
    print(f"Mean_reward: {mean_reward}")
    filename = f"./neural_dataset/{env_name}.pth"
    torch.save(qnet.state_dict(), filename)
    print(f"Model saved as {filename}")

In [7]:
GOAL_REWARD = 200
GAMMA = 0.999
REPLAY_SIZE = 100_000
BATCH_SIZE = 32
LEARNING_RATE = 1e-3
SYNC_TARGET_FRAMES = 1_000

EPSILON_DECAY_PERIOD = 75_000
EPSILON_FINAL = 2e-2

In [8]:
#ENV_NAMES += ["CartPole-v1"]
ENV_NAMES += ["LunarLander-v3"]
for i, env_name in enumerate(ENV_NAMES[-1:]):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    #replay_buffer = ReplayBuffer(DATASET_SIZE, BATCH_SIZE, device)
    buffer = ReplayBuffer(DATASET_SIZE, BATCH_SIZE, device)
    print(env_name)
    env = gym.make(env_name)
    
    qnet1 = MLP(env.observation_space.shape[0], env.action_space.n, [256,256])
    qtarget1 = MLP(env.observation_space.shape[0], env.action_space.n, [256,256])

    DQN_TRAIN(
    env = env,
    env_name = env_name,
    gamma = GAMMA,
    qnet = qnet1,
    qnet_lr = LEARNING_RATE,
    target_qnet = qtarget1,
    target_update_freq = SYNC_TARGET_FRAMES,
    replay_buffer= buffer,
    batch_size = BATCH_SIZE,
    epsilon_f = EPSILON_FINAL,
    epsilon_decay_period = EPSILON_DECAY_PERIOD,
    transitions = REPLAY_SIZE)

    #!sum_rewards_per_ep, q = run_sarsa(env, replay_buffer, DATASET_SIZE, LEARNING_RATE, GAMMA)
    ENVS_REPLAY_BUFFER.append((env_name, env, buffer))
    buffer.save_config(f"config\dataset\dqn\{env_name}.json")

LunarLander-v3


KeyboardInterrupt: 

# Treinando o Modelo

## Carregando Datasets

In [9]:
import gymnasium as gym
from util.network import ReplayBuffer


In [10]:
ENV_NAMES = ["FrozenLake-v1", "Taxi-v3", "CliffWalking-v0", "CartPole-v1", "LunarLander-v3"]

In [11]:
ENVS_REPLAY_BUFFER = []
for env_name in ENV_NAMES[:3]:
    replay_buffer = ReplayBuffer.load_config(f"config\dataset\sarsa\{env_name}.json")
    env = gym.make(env_name)
    ENVS_REPLAY_BUFFER.append((env_name, env, replay_buffer))
for env_name in ENV_NAMES[3:]:
    replay_buffer = ReplayBuffer.load_config(f"config\dataset\dqn\{env_name}.json")
    env = gym.make(env_name)
    ENVS_REPLAY_BUFFER.append((env_name, env, replay_buffer))

## Definições de Treino

In [4]:
ENV_NAMES = ["FrozenLake-v1", "Taxi-v3", "CliffWalking-v0"]


In [5]:
import argparse

In [6]:
def get_config(env_name, count_episodes, seed, gamma, tau, alpha, alpha_decay, lr, steps, hidden_size, isDiscrete):
    parser = argparse.ArgumentParser(description='RL')
    parser.add_argument("--run_name", type=str, default=f"{env_name}-DQN-CQL", help="Run name, default: CQL-DQN")
    parser.add_argument("--env", type=str, default=env_name, help="Gym environment name, default: CartPole-v0")
    parser.add_argument("--episodes", type=int, default=count_episodes, help="Number of episodes, default: 200")
    parser.add_argument("--seed", type=int, default=seed, help="Seed, default: 1")
    parser.add_argument("--steps", type=int, default=steps, help="Saves the network every x epochs, default: 25")
    
    parser.add_argument("--gamma", type=float, default=gamma, help="Saves the network every x epochs, default: 25")
    parser.add_argument("--tau", type=float, default=tau, help="Saves the network every x epochs, default: 25")
    parser.add_argument("--alpha", type=float, default=alpha, help="Saves the network every x epochs, default: 25")
    parser.add_argument("--alpha_decay", type=float, default=alpha_decay, help="Saves the network every x epochs, default: 25")
    parser.add_argument("--lr", type=float, default=lr, help="Saves the network every x epochs, default: 25")
    parser.add_argument("--hidden_size", type=list[int], default=hidden_size, help="Saves the network every x epochs, default: 25")
    parser.add_argument("--isDiscrete", type=bool, default=isDiscrete, help="Saves the network every x epochs, default: 25")
    
    args, _ = parser.parse_known_args()
    return args

In [7]:
from util.network import CQLAgent, to_one_hot, MLP
import numpy as np
import random
import torch
from collections import deque

In [8]:
def train_DQN_CQL(config, buffer):
    np.random.seed(config.seed)
    random.seed(config.seed)
    torch.manual_seed(config.seed)
    env = gym.make(config.env)

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    losses = deque(maxlen=1000)
    if config.isDiscrete:
        agent = CQLAgent(env.observation_space.n, env.action_space.n, config.tau, config.gamma, config.lr, config.alpha, config.alpha_decay, config.hidden_size, device, config.isDiscrete)
    else:
        agent = CQLAgent(env.observation_space.shape[0], env.action_space.n, config.tau, config.gamma, config.lr, config.alpha, config.alpha_decay, config.hidden_size, device, config.isDiscrete)
    for i in range(config.steps):
        loss, cql_loss, bellmann_error = agent.learn(buffer.sample_continuous())
        losses.append(loss)
        agent.alpha = max(0.3, agent.alpha - agent.alpha_decay)         
        if i % 1000 == 0:
            print(f"[TRAIN {i}] Mean Q Loss: {np.mean(losses)} ")
    
    agent.save(config, save_name="CQL-DQN")
    
    return env

In [9]:
def evaluate(config, env, model):
    total_rewards = []
    num_episodes = config.episodes
    for i in range(num_episodes):
        print(f"[Episódio {i}]")
        state, _ = env.reset()
        done = False
        episode_reward = 0
        step = 0
        while not done:
            state_tensor = torch.tensor([state], dtype=torch.long)
            state_tensor = to_one_hot(model.state_size, state_tensor)[0].float().unsqueeze(0)
            with torch.no_grad():
                q_values = model(state_tensor)
                action = torch.argmax(q_values).item()
            state, reward, done, truncated, info = env.step(action)
            episode_reward += reward
            step += 1
            if step == 10000:
                break
        total_rewards.append(episode_reward)

    print(f"Média de recompensa após {num_episodes} episódios: {sum(total_rewards) / num_episodes}")

In [10]:
# Função para extrair a tabela Q da rede neural
def extract_q_table(q_network, num_states, num_actions):
    q_table = np.zeros((num_states, num_actions))
    for state in range(num_states):
        state_tensor = torch.tensor([state], dtype=torch.long)
        state_tensor = to_one_hot(num_states, state_tensor)[0].float().unsqueeze(0)
        q_values = q_network(state_tensor).detach().numpy()
        q_table[state] = q_values
    return q_table

In [11]:
from util.qtable_helper import record_video_qtable
from util.notebook import display_videos_from_path

In [12]:

def extract_and_display(model, env_name):
    # Extrair a tabela Q da rede neural treinada
    q_table = extract_q_table(model, model.state_size, model.action_size)

    # Gravar um vídeo da política treinada
    video_path = 'videos/'  # Pasta onde os vídeos serão salvos
    video_prefix = f"cql_{env_name}"  # Prefixo para o nome do arquivo de vídeo

    # Gravar o vídeo
    record_video_qtable(env_name, q_table, episodes=2, folder=video_path, prefix=video_prefix)

    # Exibir o vídeo gravado
    display_videos_from_path(video_path, prefix=video_prefix)

In [13]:
def evaluate_Continuous(config, env, model):
    total_rewards = []
    num_episodes = config.episodes
    for i in range(num_episodes):
        print(f"[Episódio {i}]")
        state, _ = env.reset()
        done = False
        episode_reward = 0.0
        step = 0
        while not done:
            state_v = torch.tensor(state, dtype=torch.float32)
            state_v = state_v.unsqueeze(0)  # Adiciona dimensão de batch como eixo 0 (e.g. transforma uma lista [a,b,c] em [[a,b,c]])
            with torch.no_grad():
                q_vals_v = model(state_v)
                _, act_v = torch.max(q_vals_v, dim=1)
            action = int(act_v.item())


            state, reward, done, truncated, info = env.step(action)
            episode_reward += reward
            step += 1
            if step == 10000:
                break
        total_rewards.append(episode_reward)

    print(f"Média de recompensa após {num_episodes} episódios: {sum(total_rewards) / num_episodes}")

In [14]:
# Função para renderizar o ambiente com o modelo treinado
import time
def render_environment(model, env_name, device="cpu", episodes=5, max_steps=100000):
    env = gym.make(env_name, render_mode="human")  # Modo "human" para renderização
    for episode in range(episodes):
        state, _ = env.reset()
        done = False
        episode_reward = 0
        step = 0

        while not done and step < max_steps:
            # Renderiza o ambiente
            env.render()
            time.sleep(0.02)  # Adiciona um pequeno delay para visualização

            # Converte o estado para um tensor e passa pelo modelo
            state_tensor = torch.tensor(state, dtype=torch.float32).unsqueeze(0).to(device)
            with torch.no_grad():
                q_values = model(state_tensor)
                action = torch.argmax(q_values).item()

            # Executa a ação no ambiente
            state, reward, done, truncated, info = env.step(action)
            episode_reward += reward
            step += 1

            if done or truncated:
                print(f"Episódio {episode + 1}: Recompensa = {episode_reward}, Passos = {step}")
                break

    env.close()

## FrozenLake

In [ ]:
COUNT_EPISODES = 50
SEED = 777
BATCH_SIZE = 32
STEPS = 100_000
GAMMA = 0.95 # Fator de desconto
TAU = 1e-1      # Taxa de atualização da target_network
ALPHA = 0.9      # Peso do termo CQL
APLHA_DECAY = ALPHA / (0.85 * STEPS)
LEARNING_RATE = 3e-4  # Taxa de aprendizado para o otimizador
HIDDEN_SIZE = [64, 32]
ISDISCRETE = True
# Extra:

In [16]:
ENVS_REPLAY_BUFFER[0][2].batch_size = BATCH_SIZE

In [17]:
config = get_config(ENVS_REPLAY_BUFFER[0][0], COUNT_EPISODES, SEED, GAMMA, TAU, ALPHA, APLHA_DECAY, LEARNING_RATE, STEPS, HIDDEN_SIZE, ISDISCRETE)

In [18]:
env = train_DQN_CQL(config, ENVS_REPLAY_BUFFER[0][2])

d:\Luiz\Faculdade\RL\OffRL\venv\lib\site-packages\torch\nn\modules\loss.py:1042: UserWarning: Using a target size (torch.Size([32, 32])) that is different to the input size (torch.Size([32])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.smooth_l1_loss(input, target, reduction=self.reduction, beta=self.beta)


[TRAIN 0] Mean Q Loss: 1.426167368888855 
[TRAIN 1000] Mean Q Loss: 0.7519247961044312 
[TRAIN 2000] Mean Q Loss: 0.5954154410660267 
[TRAIN 3000] Mean Q Loss: 0.5716989253461361 
[TRAIN 4000] Mean Q Loss: 0.5881283572316169 
[TRAIN 5000] Mean Q Loss: 0.5732272872030735 
[TRAIN 6000] Mean Q Loss: 0.5694553360939026 
[TRAIN 7000] Mean Q Loss: 0.5718850112110376 
[TRAIN 8000] Mean Q Loss: 0.5789433391094208 
[TRAIN 9000] Mean Q Loss: 0.5817794059365988 
[TRAIN 10000] Mean Q Loss: 0.568873725026846 
[TRAIN 11000] Mean Q Loss: 0.5740846926271915 
[TRAIN 12000] Mean Q Loss: 0.5668653095513583 
[TRAIN 13000] Mean Q Loss: 0.5768260368704796 
[TRAIN 14000] Mean Q Loss: 0.5620772553384304 
[TRAIN 15000] Mean Q Loss: 0.5545780383050442 
[TRAIN 16000] Mean Q Loss: 0.5780531690567732 
[TRAIN 17000] Mean Q Loss: 0.5622173122316599 
[TRAIN 18000] Mean Q Loss: 0.5518166505098343 
[TRAIN 19000] Mean Q Loss: 0.5479077635258436 
[TRAIN 20000] Mean Q Loss: 0.5654774218499661 
[TRAIN 21000] Mean Q Loss: 0

KeyboardInterrupt: 

In [ ]:
import torch

In [ ]:
env = ENVS_REPLAY_BUFFER[0][1]
model = MLP(env.observation_space.n, env.action_space.n, HIDDEN_SIZE)
model.load_state_dict(torch.load(f"trained_models/{ENVS_REPLAY_BUFFER[0][0]}CQL-DQN.pth"))
model.eval()  # Definir para modo de avaliação

MLP(
  (layers): Sequential(
    (0): Linear(in_features=16, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=32, bias=True)
    (3): ReLU()
    (4): Linear(in_features=32, out_features=4, bias=True)
  )
)

In [ ]:
evaluate(config, env, model)

[Episódio 0]
[Episódio 1]
[Episódio 2]
[Episódio 3]
[Episódio 4]
[Episódio 5]
[Episódio 6]
[Episódio 7]
[Episódio 8]
[Episódio 9]
[Episódio 10]
[Episódio 11]
[Episódio 12]
[Episódio 13]
[Episódio 14]
[Episódio 15]
[Episódio 16]
[Episódio 17]
[Episódio 18]
[Episódio 19]
[Episódio 20]
[Episódio 21]
[Episódio 22]
[Episódio 23]
[Episódio 24]
[Episódio 25]
[Episódio 26]
[Episódio 27]
[Episódio 28]
[Episódio 29]
[Episódio 30]
[Episódio 31]
[Episódio 32]
[Episódio 33]
[Episódio 34]
[Episódio 35]
[Episódio 36]
[Episódio 37]
[Episódio 38]
[Episódio 39]
[Episódio 40]
[Episódio 41]
[Episódio 42]
[Episódio 43]
[Episódio 44]
[Episódio 45]
[Episódio 46]
[Episódio 47]
[Episódio 48]
[Episódio 49]
Média de recompensa após 50 episódios: 0.06


d:\Luiz\Faculdade\RL\OffRL\util\network.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  states = torch.tensor(states, dtype=torch.long)  # Garante que é tensor de inteiros


In [ ]:
extract_and_display(model, ENVS_REPLAY_BUFFER[0][0])

d:\Luiz\Faculdade\RL\OffRL\venv\lib\site-packages\gymnasium\wrappers\rendering.py:283: UserWarning: WARN: Overwriting existing videos at d:\Luiz\Faculdade\RL\OffRL\videos folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


## Taxi

In [32]:
from util.network import TaxiNet

In [ ]:
COUNT_EPISODES = 50
SEED = 777
BATCH_SIZE = 256
STEPS = 300_000
GAMMA = 0.97  # Fator de desconto
TAU = 1e-2      # Taxa de atualização da target_network
ALPHA = 1.      # Peso do termo CQL
APLHA_DECAY = ALPHA / (0.85 * STEPS)
LEARNING_RATE = 5e-4  # Taxa de aprendizado para o otimizador
HIDDEN_SIZE = [512, 256]
ISDISCRETE = True
# Extra:

In [34]:
ENVS_REPLAY_BUFFER[1][2].batch_size = BATCH_SIZE

In [ ]:
config = get_config(ENVS_REPLAY_BUFFER[1][0], COUNT_EPISODES, SEED, GAMMA, TAU, ALPHA, APLHA_DECAY, LEARNING_RATE, STEPS, HIDDEN_SIZE)

In [36]:
env = train_DQN_CQL(config, ENVS_REPLAY_BUFFER[1][2], TaxiNet)

[TRAIN 0] Q Loss: 2.9946062564849854 
[TRAIN 1000] Q Loss: 1.9792652130126953 
[TRAIN 2000] Q Loss: 2.1473395824432373 
[TRAIN 3000] Q Loss: 2.3356688022613525 
[TRAIN 4000] Q Loss: 2.3304543495178223 
[TRAIN 5000] Q Loss: 2.3218111991882324 
[TRAIN 6000] Q Loss: 2.379727363586426 
[TRAIN 7000] Q Loss: 2.350546360015869 
[TRAIN 8000] Q Loss: 2.3016459941864014 
[TRAIN 9000] Q Loss: 2.207245111465454 
[TRAIN 10000] Q Loss: 1.939531922340393 
[TRAIN 11000] Q Loss: 2.1184892654418945 
[TRAIN 12000] Q Loss: 2.338794231414795 
[TRAIN 13000] Q Loss: 2.127078056335449 
[TRAIN 14000] Q Loss: 1.708202838897705 
[TRAIN 15000] Q Loss: 2.168572187423706 
[TRAIN 16000] Q Loss: 1.839595079421997 
[TRAIN 17000] Q Loss: 2.2350282669067383 
[TRAIN 18000] Q Loss: 1.812806248664856 
[TRAIN 19000] Q Loss: 1.7697908878326416 
[TRAIN 20000] Q Loss: 1.761971116065979 
[TRAIN 21000] Q Loss: 2.068164825439453 
[TRAIN 22000] Q Loss: 1.9779565334320068 
[TRAIN 23000] Q Loss: 1.878171682357788 
[TRAIN 24000] Q Lo

In [37]:
import torch

In [38]:
env = ENVS_REPLAY_BUFFER[1][1]
model = TaxiNet(env.observation_space.n, env.action_space.n, HIDDEN_SIZE)
model.load_state_dict(torch.load(f"trained_models/{ENVS_REPLAY_BUFFER[1][0]}CQL-DQN.pth"))
model.eval()  # Definir para modo de avaliação

TaxiNet(
  (fc1): Linear(in_features=500, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=256, bias=True)
  (fc4): Linear(in_features=256, out_features=128, bias=True)
  (fc5): Linear(in_features=128, out_features=6, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
)

In [39]:
evaluate(config, env, model)

[Episódio 0]
[Episódio 1]
[Episódio 2]
[Episódio 3]
[Episódio 4]
[Episódio 5]
[Episódio 6]
[Episódio 7]
[Episódio 8]
[Episódio 9]
[Episódio 10]
[Episódio 11]
[Episódio 12]
[Episódio 13]
[Episódio 14]
[Episódio 15]
[Episódio 16]
[Episódio 17]
[Episódio 18]
[Episódio 19]
[Episódio 20]
[Episódio 21]
[Episódio 22]
[Episódio 23]
[Episódio 24]
[Episódio 25]
[Episódio 26]
[Episódio 27]
[Episódio 28]
[Episódio 29]
[Episódio 30]
[Episódio 31]
[Episódio 32]
[Episódio 33]
[Episódio 34]
[Episódio 35]
[Episódio 36]
[Episódio 37]
[Episódio 38]
[Episódio 39]
[Episódio 40]
[Episódio 41]
[Episódio 42]
[Episódio 43]
[Episódio 44]
[Episódio 45]
[Episódio 46]
[Episódio 47]
[Episódio 48]
[Episódio 49]
Média de recompensa após 50 episódios: -3995.18


In [40]:
extract_and_display(model, ENVS_REPLAY_BUFFER[1][0])

d:\Luiz\Faculdade\RL\OffRL\venv\Lib\site-packages\gymnasium\wrappers\rendering.py:283: UserWarning: WARN: Overwriting existing videos at d:\Luiz\Faculdade\RL\OffRL\videos folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


## Cliffwalking

In [ ]:
COUNT_EPISODES = 50
SEED = 777
BATCH_SIZE = 128
STEPS = 200_000
GAMMA = 0.98  # Fator de desconto
TAU = 5e-3      # Taxa de atualização da target_network
ALPHA = 1      # Peso do termo CQL
APLHA_DECAY = ALPHA / (0.85 * STEPS)
LEARNING_RATE = 5e-4  # Taxa de aprendizado para o otimizador
HIDDEN_SIZE = [256, 128, 64]  # Tamanho da camada oculta
ISDISCRETE = True
# Extra:

In [78]:
ENVS_REPLAY_BUFFER[2][2].batch_size = BATCH_SIZE

In [ ]:
config = get_config(ENVS_REPLAY_BUFFER[2][0], COUNT_EPISODES, SEED, GAMMA, TAU, ALPHA, APLHA_DECAY, LEARNING_RATE, STEPS, HIDDEN_SIZE, ISDISCRETE)

In [80]:
env = train_DQN_CQL(config, ENVS_REPLAY_BUFFER[2][2])

[TRAIN 0] Mean Q Loss: 2.611858367919922 
[TRAIN 1000] Mean Q Loss: 1.9560353758335114 
[TRAIN 2000] Mean Q Loss: 1.9201923592090606 
[TRAIN 3000] Mean Q Loss: 1.981021082997322 
[TRAIN 4000] Mean Q Loss: 2.0206536754369737 
[TRAIN 5000] Mean Q Loss: 2.0379670689105986 
[TRAIN 6000] Mean Q Loss: 2.03328164601326 
[TRAIN 7000] Mean Q Loss: 2.0815190527439116 
[TRAIN 8000] Mean Q Loss: 2.0760167702436445 
[TRAIN 9000] Mean Q Loss: 2.0571795363426206 
[TRAIN 10000] Mean Q Loss: 2.102876945972443 
[TRAIN 11000] Mean Q Loss: 2.0759065252542497 
[TRAIN 12000] Mean Q Loss: 2.0772983828783036 
[TRAIN 13000] Mean Q Loss: 2.052657237172127 
[TRAIN 14000] Mean Q Loss: 2.073210382938385 
[TRAIN 15000] Mean Q Loss: 2.0741584622859954 
[TRAIN 16000] Mean Q Loss: 2.048782295227051 
[TRAIN 17000] Mean Q Loss: 2.060649689912796 
[TRAIN 18000] Mean Q Loss: 2.0661960541009905 
[TRAIN 19000] Mean Q Loss: 2.0721865670681 
[TRAIN 20000] Mean Q Loss: 2.0771966969966886 
[TRAIN 21000] Mean Q Loss: 2.014461856

In [81]:
import torch

In [83]:
env = ENVS_REPLAY_BUFFER[2][1]
model = MLP(env.observation_space.n, env.action_space.n, HIDDEN_SIZE)
model.load_state_dict(torch.load(f"trained_models/{ENVS_REPLAY_BUFFER[2][0]}CQL-DQN.pth"))
model.eval()  # Definir para modo de avaliação

MLP(
  (layers): Sequential(
    (0): Linear(in_features=48, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=128, bias=True)
    (3): ReLU()
    (4): Linear(in_features=128, out_features=64, bias=True)
    (5): ReLU()
    (6): Linear(in_features=64, out_features=4, bias=True)
  )
)

In [84]:
evaluate(config, env, model)

[Episódio 0]
[Episódio 1]
[Episódio 2]
[Episódio 3]
[Episódio 4]
[Episódio 5]
[Episódio 6]
[Episódio 7]
[Episódio 8]
[Episódio 9]
[Episódio 10]
[Episódio 11]
[Episódio 12]
[Episódio 13]
[Episódio 14]
[Episódio 15]
[Episódio 16]
[Episódio 17]
[Episódio 18]
[Episódio 19]
[Episódio 20]
[Episódio 21]
[Episódio 22]
[Episódio 23]
[Episódio 24]
[Episódio 25]
[Episódio 26]
[Episódio 27]
[Episódio 28]
[Episódio 29]
[Episódio 30]
[Episódio 31]
[Episódio 32]
[Episódio 33]
[Episódio 34]
[Episódio 35]
[Episódio 36]
[Episódio 37]
[Episódio 38]
[Episódio 39]
[Episódio 40]
[Episódio 41]
[Episódio 42]
[Episódio 43]
[Episódio 44]
[Episódio 45]
[Episódio 46]
[Episódio 47]
[Episódio 48]
[Episódio 49]
Média de recompensa após 50 episódios: -13.0


In [85]:
extract_and_display(model, ENVS_REPLAY_BUFFER[2][0])

d:\Luiz\Faculdade\RL\OffRL\venv\lib\site-packages\gymnasium\wrappers\rendering.py:283: UserWarning: WARN: Overwriting existing videos at d:\Luiz\Faculdade\RL\OffRL\videos folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


## CartPole

In [ ]:
COUNT_EPISODES = 50
SEED = 777
BATCH_SIZE = 256
STEPS = 50_000
GAMMA = 0.99  # Fator de desconto
TAU = 5e-3      # Taxa de atualização da target_network
ALPHA = 0.5     # Peso do termo CQL
APLHA_DECAY = ALPHA / (0.85 * STEPS)
LEARNING_RATE = 1e-4  # Taxa de aprendizado para o otimizador
HIDDEN_SIZE = [256, 128]
ISDISCRETE = False
# Extra:

In [29]:
ENVS_REPLAY_BUFFER[3][2].batch_size = BATCH_SIZE

In [ ]:
config = get_config(ENVS_REPLAY_BUFFER[3][0], COUNT_EPISODES, SEED, GAMMA, TAU, ALPHA, APLHA_DECAY, LEARNING_RATE, STEPS, HIDDEN_SIZE, ISDISCRETE)

In [31]:
ENVS_REPLAY_BUFFER[3][2].memory

deque([Experience(state=[0.0444495715200901, -0.027678364887833595, 0.00578987505286932, 0.042084816843271255], action=np.int64(0), reward=1.0, done=False, next_state=[0.04389600455760956, -0.22288286685943604, 0.0066315713338553905, 0.33658885955810547]),
       Experience(state=[0.04389600455760956, -0.22288286685943604, 0.0066315713338553905, 0.33658885955810547], action=np.int64(0), reward=1.0, done=False, next_state=[0.03943834826350212, -0.4180985689163208, 0.013363348320126534, 0.6313556432723999]),
       Experience(state=[0.03943834826350212, -0.4180985689163208, 0.013363348320126534, 0.6313556432723999], action=np.int64(0), reward=1.0, done=False, next_state=[0.031076375395059586, -0.613404393196106, 0.02599046193063259, 0.9282169342041016]),
       Experience(state=[0.031076375395059586, -0.613404393196106, 0.02599046193063259, 0.9282169342041016], action=np.int64(1), reward=1.0, done=False, next_state=[0.018808288499712944, -0.4186427891254425, 0.04455479979515076, 0.643813

In [32]:
env = train_DQN_CQL(config, ENVS_REPLAY_BUFFER[3][2])

[TRAIN 0] Mean Q Loss: 1.0043326616287231 
[TRAIN 1000] Mean Q Loss: 0.7007717928886413 
[TRAIN 2000] Mean Q Loss: 0.7205777180194854 
[TRAIN 3000] Mean Q Loss: 0.7634395972490311 
[TRAIN 4000] Mean Q Loss: 0.80464292293787 
[TRAIN 5000] Mean Q Loss: 0.8426218659281731 
[TRAIN 6000] Mean Q Loss: 0.8810978090763092 
[TRAIN 7000] Mean Q Loss: 0.9137464919686318 
[TRAIN 8000] Mean Q Loss: 0.9625849985480308 
[TRAIN 9000] Mean Q Loss: 0.984437583208084 
[TRAIN 10000] Mean Q Loss: 1.015138298213482 
[TRAIN 11000] Mean Q Loss: 1.0550723363757133 
[TRAIN 12000] Mean Q Loss: 1.084304897725582 
[TRAIN 13000] Mean Q Loss: 1.1074400176405907 
[TRAIN 14000] Mean Q Loss: 1.1418750166296958 
[TRAIN 15000] Mean Q Loss: 1.1637242393493652 
[TRAIN 16000] Mean Q Loss: 1.1803629774451256 
[TRAIN 17000] Mean Q Loss: 1.2117309328913688 
[TRAIN 18000] Mean Q Loss: 1.2242265382409097 
[TRAIN 19000] Mean Q Loss: 1.2518561643958093 
[TRAIN 20000] Mean Q Loss: 1.2589832950234414 
[TRAIN 21000] Mean Q Loss: 1.29

In [33]:
import torch

In [34]:
env = ENVS_REPLAY_BUFFER[3][1]
model = MLP(env.observation_space.shape[0], env.action_space.n, HIDDEN_SIZE)
model.load_state_dict(torch.load(f"trained_models/{ENVS_REPLAY_BUFFER[3][0]}CQL-DQN.pth"))
model.eval()  # Definir para modo de avaliação

MLP(
  (layers): Sequential(
    (0): Linear(in_features=4, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=128, bias=True)
    (3): ReLU()
    (4): Linear(in_features=128, out_features=2, bias=True)
  )
)

In [37]:
evaluate_Continuous(config, env, model)

[Episódio 0]
[Episódio 1]
[Episódio 2]
[Episódio 3]
[Episódio 4]
[Episódio 5]
[Episódio 6]
[Episódio 7]
[Episódio 8]
[Episódio 9]
[Episódio 10]
[Episódio 11]
[Episódio 12]
[Episódio 13]
[Episódio 14]
[Episódio 15]
[Episódio 16]
[Episódio 17]
[Episódio 18]
[Episódio 19]
[Episódio 20]
[Episódio 21]
[Episódio 22]
[Episódio 23]
[Episódio 24]
[Episódio 25]
[Episódio 26]
[Episódio 27]
[Episódio 28]
[Episódio 29]
[Episódio 30]
[Episódio 31]
[Episódio 32]
[Episódio 33]
[Episódio 34]
[Episódio 35]
[Episódio 36]
[Episódio 37]
[Episódio 38]
[Episódio 39]
[Episódio 40]
[Episódio 41]
[Episódio 42]
[Episódio 43]
[Episódio 44]
[Episódio 45]
[Episódio 46]
[Episódio 47]
[Episódio 48]
[Episódio 49]
Média de recompensa após 50 episódios: 269.86


In [48]:
#extract_and_display(model, ENVS_REPLAY_BUFFER[3][0])
render_environment(model, ENVS_REPLAY_BUFFER[3][0])

Episódio 1: Recompensa = 316.0, Passos = 316
Episódio 2: Recompensa = 333.0, Passos = 333
Episódio 3: Recompensa = 361.0, Passos = 361
Episódio 4: Recompensa = 233.0, Passos = 233
Episódio 5: Recompensa = 164.0, Passos = 164


## LunarLander

In [ ]:
COUNT_EPISODES = 50
SEED = 777
BATCH_SIZE = 256
STEPS = 50_000
GAMMA = 0.99  # Fator de desconto
TAU = 5e-3      # Taxa de atualização da target_network
ALPHA = 0.5     # Peso do termo CQL
APLHA_DECAY = ALPHA / (0.85 * STEPS)
LEARNING_RATE = 1e-4  # Taxa de aprendizado para o otimizador
HIDDEN_SIZE = [256, 256]
ISDISCRETE = False
# Extra:

In [26]:
ENVS_REPLAY_BUFFER[4][2].batch_size = BATCH_SIZE

In [ ]:
config = get_config(ENVS_REPLAY_BUFFER[4][0], COUNT_EPISODES, SEED, GAMMA, TAU, ALPHA, APLHA_DECAY, LEARNING_RATE, STEPS, HIDDEN_SIZE, ISDISCRETE)

In [28]:
env = train_DQN_CQL(config, ENVS_REPLAY_BUFFER[4][2])

[TRAIN 0] Mean Q Loss: 1.964367389678955 
[TRAIN 1000] Mean Q Loss: 2.0619446742534637 
[TRAIN 2000] Mean Q Loss: 1.9862973977327347 
[TRAIN 3000] Mean Q Loss: 1.9522753010988236 
[TRAIN 4000] Mean Q Loss: 1.9664960606098174 
[TRAIN 5000] Mean Q Loss: 1.9730189980268478 
[TRAIN 6000] Mean Q Loss: 2.0195518946647644 
[TRAIN 7000] Mean Q Loss: 2.0561872282028197 
[TRAIN 8000] Mean Q Loss: 2.1188316057920455 
[TRAIN 9000] Mean Q Loss: 2.2026632455587385 
[TRAIN 10000] Mean Q Loss: 2.289009927511215 
[TRAIN 11000] Mean Q Loss: 2.379326355934143 
[TRAIN 12000] Mean Q Loss: 2.5046861090660095 
[TRAIN 13000] Mean Q Loss: 2.621540540933609 
[TRAIN 14000] Mean Q Loss: 2.7828504470586775 
[TRAIN 15000] Mean Q Loss: 2.9515529201030732 
[TRAIN 16000] Mean Q Loss: 3.116341412305832 
[TRAIN 17000] Mean Q Loss: 3.3376150877475737 
[TRAIN 18000] Mean Q Loss: 3.5541613607406615 
[TRAIN 19000] Mean Q Loss: 3.761573856830597 
[TRAIN 20000] Mean Q Loss: 3.948637573957443 
[TRAIN 21000] Mean Q Loss: 4.1972

In [29]:
import torch

In [30]:
env = ENVS_REPLAY_BUFFER[4][1]
model = MLP(env.observation_space.shape[0], env.action_space.n, HIDDEN_SIZE)
model.load_state_dict(torch.load(f"trained_models/{ENVS_REPLAY_BUFFER[4][0]}CQL-DQN.pth"))
model.eval()  # Definir para modo de avaliação

MLP(
  (layers): Sequential(
    (0): Linear(in_features=8, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=128, bias=True)
    (3): ReLU()
    (4): Linear(in_features=128, out_features=4, bias=True)
  )
)

In [31]:
evaluate_Continuous(config, env, model)

[Episódio 0]
[Episódio 1]
[Episódio 2]
[Episódio 3]
[Episódio 4]
[Episódio 5]
[Episódio 6]
[Episódio 7]
[Episódio 8]
[Episódio 9]
[Episódio 10]
[Episódio 11]
[Episódio 12]
[Episódio 13]
[Episódio 14]
[Episódio 15]
[Episódio 16]
[Episódio 17]
[Episódio 18]
[Episódio 19]
[Episódio 20]
[Episódio 21]
[Episódio 22]
[Episódio 23]
[Episódio 24]
[Episódio 25]
[Episódio 26]
[Episódio 27]
[Episódio 28]
[Episódio 29]
[Episódio 30]
[Episódio 31]
[Episódio 32]
[Episódio 33]
[Episódio 34]
[Episódio 35]
[Episódio 36]
[Episódio 37]
[Episódio 38]
[Episódio 39]
[Episódio 40]
[Episódio 41]
[Episódio 42]
[Episódio 43]
[Episódio 44]
[Episódio 45]
[Episódio 46]
[Episódio 47]
[Episódio 48]
[Episódio 49]
Média de recompensa após 50 episódios: -357.11909701857314


In [ ]:
#extract_and_display(model, ENVS_REPLAY_BUFFER[3][0])
render_environment(model, ENVS_REPLAY_BUFFER[4][0])

Episódio 1: Recompensa = -584.9578590209794, Passos = 598
Episódio 2: Recompensa = -310.7386188899749, Passos = 552
Episódio 3: Recompensa = -308.5864007015114, Passos = 824


KeyboardInterrupt: 

: 